In [6]:
ls Reranker/src/reranker

__init__.py   arguments.py  dist/        trainer.py
__pycache__/  data.py       modeling.py


In [1]:
from Reranker.src.reranker.arguments import ModelArguments, DataArguments, \
    RerankerTrainingArguments as TrainingArguments

In [2]:
from transformers import AutoTokenizer



In [3]:
from reranker import Reranker 
from transformers import AutoTokenizer

In [48]:
from typing import Optional

import torch
import torch.functional as F
import copy
from transformers import AutoModelForSequenceClassification, AutoTokenizer,\
    PreTrainedModel, PreTrainedTokenizer, GPTNeoForSequenceClassification

from transformers.modeling_outputs import SequenceClassifierOutput, BaseModelOutputWithPooling
from torch import nn
import torch.distributed as dist

class RerankerForInference(nn.Module):
    def __init__(
            self,
            hf_model: Optional[PreTrainedModel] = None,
            tokenizer: Optional[PreTrainedTokenizer] = None
    ):
        super().__init__()
        self.hf_model = hf_model
        self.tokenizer = tokenizer

    def tokenize(self, *args, **kwargs):
        return self.tokenizer(*args, **kwargs)

    def forward(self, batch):
        return self.hf_model(**batch)

    @classmethod
    def from_pretrained(cls, pretrained_model_name_or_path: str):
        hf_model = GPTNeoForSequenceClassification.from_pretrained(
            pretrained_model_name_or_path,num_labels=1)
        hf_tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M",    
                            bos_token="<|startoftext|>",
                            eos_token="<|endoftext|>",
                            pad_token="<|pad|>")
        hf_model.cuda()
        hf_model.eval()
        return cls(hf_model, hf_tokenizer)

    def load_pretrained_model(self, pretrained_model_name_or_path, *model_args, **kwargs):
        self.hf_model = AutoModelForSequenceClassification.from_pretrained(
            pretrained_model_name_or_path, *model_args, **kwargs
        )

    def load_pretrained_tokenizer(self, pretrained_model_name_or_path, *inputs, **kwargs):
        self.tokenizer = AutoTokenizer.from_pretrained(
            pretrained_model_name_or_path, *inputs, **kwargs
        )

In [49]:
data_args ={
    "train_dir"
}

In [50]:
rk = RerankerForInference.from_pretrained("results/checkpoint-140000/") 

Some weights of GPTNeoForSequenceClassification were not initialized from the model checkpoint at results/checkpoint-140000/ and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
# rk.hf_model

In [20]:
import json

with open('all_gpt_combined.json', 'r') as json_file:
    json_c = [json.loads(i) for i in list(json_file)][0]

In [21]:
len(json_c)

7128

In [22]:
exa = json_c[0]

In [23]:
exa["score"]

0.5

In [24]:
exa["best_match"]

'cases i'

In [25]:
exa["real"]

'cases b'

In [26]:
str(hash(1))

'1'

In [33]:
def get_embed(exa):
    if exa["score"]!=0:
        positive =  [exa["best_match"],exa["real"]]
        negative = [i for i in exa["all"] if i not in positive]
    else:
        negative = exa["all"]
        positive =  [exa["best_match"]]

    positive_token = [rk.tokenize(i, return_tensors='pt') for i in positive]
    negative_token = [rk.tokenize(i, return_tensors='pt') for i in negative]
    prompt = exa["prompt"].replace("GOAL","<GOAL>").replace("PROOFSTEP","<PROOFSTEP>")
    prompt_token = rk.tokenize(prompt, return_tensors='pt').input_ids
    idi = str(hash(prompt))
    return { "qry": {  "qid": idi,  "query": prompt_token, }, "pos": [ {"pid": str(hash(i)),  "passage": j  } for i,j in zip(positive,positive_token)]        ,"neg": [ {"pid": str(hash(i)),  "passage": j  } for i,j in zip(negative,negative_token)]}

In [34]:
from tqdm import tqdm

In [102]:
json_c_train = []

for i in tqdm(json_c):
    json_c_train.append(get_embed(i))
    

100% 7128/7128 [00:13<00:00, 519.24it/s]


In [39]:
json_c[0]

{'best_match': 'cases i',
 'score': 0.5,
 'real': 'cases b',
 'all': ['cases i',
  "apply b.reverse_corec'",
  'apply b.append_right_',
  'apply b',
  'apply fin',
  'dsimp [read] at h',
  'apply_instance',
  'simp! * at *',
  'dsimp only [read, dsimp] at h',
  "set i := b.reverse_core i with h'",
  'apply b.append_left_inj',
  'rw [size, h]',
  '⊢ b.read',
  'apply mem',
  'simp [append_right]',
  'delta read',
  'simp [size, h]',
  'subst i',
  "rw [array.read_write', array.write'_write, buffer.write_mem, array.write_mem, h]",
  'rw [←append_right_inj]',
  'simp',
  'dsimp [read]',
  "apply b.append_right'",
  'set_of_eq_eq_succ_of_eq',
  "convert b.read'",
  'contradiction',
  'rw [←append_right_inverse_iff, ←',
  "apply b.read'_eq_of_mem",
  '_match : ∀ (_a _a_1 :append (list α) ⟨i,',
  'apply b.append_right',
  'apply b.append_left_injective',
  '_inst : inhabited α :=',
  'apply b.reverse_core_eq',
  'convert read_inj b',
  '⊢ b.reverse_'],
 'prompt': "GOAL α : Type u,\t_inst_1 :

In [ ]:
inputs = rk.tokenize(json_c["promp"], 'it is cold today in new york', return_tensors='pt')

In [38]:
json_c_train[0]

{'qry': {'qid': '-6574462426148653399',
  'query': tensor([[   27, 11230,  1847,    29, 26367,  1058,  5994,   334,    11,   197,
              62,  8625,    62,    16,  1058, 30671, 26367,    11,   197,    65,
            1058, 11876, 26367,    11,   197,    72,  1058,  2343,   226,   243,
              11,   197,    71,  1058,  1312,  1279,   275,    13,  7857,   197,
             158,   232,    95,   275,    13,   961,  2343,   253,   101,    72,
              11,   289,   158,   253,   102,   796,   275,    13,   961,     6,
            1312,   198,  1279,  4805,  6684,    37, 42135,    29,   220]])},
 'pos': [{'pid': '8592327939438459529',
   'passage': {'input_ids': tensor([[33964,  1312]]), 'attention_mask': tensor([[1, 1]])}},
  {'pid': '-6997269615945621261',
   'passage': {'input_ids': tensor([[33964,   275]]), 'attention_mask': tensor([[1, 1]])}}],
 'neg': [{'pid': '-1059230821898650523',
   'passage': {'input_ids': tensor([[39014,   275,    13, 50188,    62,  7295,    66,  

In [237]:
from sklearn.model_selection import train_test_split

In [238]:
X_train,x_dev = train_test_split(json_c_train,test_size= 0.15)

In [239]:
len(X_train)

6058

In [240]:
X_train[0]

{'qry': {'qid': '918643313112262782',
  'query': [27,
   11230,
   1847,
   29,
   299,
   1058,
   2343,
   226,
   243,
   11,
   197,
   37,
   1058,
   2099,
   35138,
   357,
   77,
   1343,
   352,
   8,
   15168,
   5994,
   334,
   11,
   197,
   62,
   8625,
   62,
   16,
   1058,
   285,
   85,
   12543,
   2715,
   376,
   11,
   197,
   80,
   1058,
   285,
   85,
   80,
   79,
   69,
   376,
   11,
   197,
   17394,
   1058,
   2099,
   35138,
   299,
   11,
   197,
   81,
   1058,
   285,
   85,
   80,
   79,
   69,
   13,
   1073,
   13049,
   376,
   26367,
   15168,
   285,
   85,
   80,
   79,
   69,
   13,
   1073,
   13049,
   376,
   26367,
   15168,
   8772,
   11,
   197,
   71,
   1058,
   18872,
   222,
   357,
   87,
   331,
   1058,
   285,
   85,
   80,
   79,
   69,
   13,
   1073,
   13049,
   376,
   26367,
   828,
   374,
   2124,
   331,
   15168,
   285,
   85,
   12543,
   2715,
   13,
   26282,
   81,
   357,
   17394,
   13,
   2411,
   62,
   12957

In [273]:
# with open('train.json', 'w') as fout:
#     json.dump(X_train , fout)
    
with open('data/train/train.json', 'w') as outfile:
    for entry in X_train:
        json.dump(entry, outfile)
        outfile.write('\n')

In [274]:
# with open('dev.json', 'w') as fout:
#     json.dump(x_dev , fout)
    
with open('data/dev/dev.json', 'w') as outfile:
    for entry in x_dev:
        json.dump(entry, outfile)
        outfile.write('\n')

In [13]:
!du -sh *

432K	Reranker
6.2M	all_gpt-ada.json
11M	all_gpt_combined.json
538M	class
26M	data
562M	files_upload
7.9M	gdrive
20K	gptNEO_eval.ipynb
1012K	json_data_0.json
7.9M	json_data_1.json
4.3M	json_data__hard_ones.json
740K	neo-Copy1.ipynb
908K	neo.ipynb
0	onstart.log
4.0K	onstart.sh
116K	reranker.ipynb
11G	results
0	test.json
0	tmp_trainer


In [90]:
def get_best_ranking(each):
    test = each["prompt"].replace("GOAL","<GOAL>").replace("PROOFSTEP","<PROOFSTEP>")
    
    results = [i for i in each["all"]]
    l = []
    for i in results+[each["real"]]:
        inputs = rk.tokenize(test, i, return_tensors='pt')
        inputs.to(torch.device("cuda:0"))
        score = rk(inputs).logits
        l.append((score.cpu().detach().numpy()[0][0],i))
    l.sort(key=lambda x:x[0])
    l.reverse()
    l_e = [i[1] for i in l]
    return l_e.index(each["real"]),l_e.index(each["best_match"])

In [103]:
resi_search = []
    
for i in tqdm(json_c[:1000]):
    resi_search.append(get_best_ranking(i))

100% 1000/1000 [03:01<00:00,  5.52it/s]


In [104]:
resi_search

[(15, 23),
 (17, 25),
 (16, 15),
 (15, 5),
 (13, 20),
 (9, 4),
 (20, 20),
 (2, 12),
 (11, 16),
 (0, 6),
 (15, 15),
 (2, 12),
 (8, 8),
 (2, 8),
 (2, 3),
 (8, 8),
 (14, 14),
 (0, 6),
 (1, 1),
 (11, 11),
 (13, 13),
 (4, 10),
 (3, 3),
 (16, 16),
 (5, 5),
 (10, 1),
 (5, 5),
 (5, 5),
 (6, 19),
 (6, 6),
 (5, 5),
 (8, 21),
 (3, 10),
 (4, 11),
 (10, 16),
 (6, 6),
 (8, 12),
 (14, 17),
 (21, 9),
 (2, 15),
 (21, 12),
 (5, 6),
 (7, 6),
 (17, 14),
 (2, 15),
 (21, 12),
 (12, 4),
 (3, 3),
 (22, 2),
 (7, 0),
 (15, 15),
 (15, 10),
 (14, 10),
 (11, 13),
 (1, 11),
 (20, 16),
 (1, 12),
 (11, 12),
 (15, 16),
 (7, 7),
 (12, 12),
 (5, 5),
 (14, 15),
 (8, 8),
 (6, 10),
 (15, 4),
 (9, 14),
 (15, 2),
 (12, 17),
 (14, 14),
 (15, 15),
 (11, 6),
 (5, 6),
 (14, 14),
 (12, 13),
 (9, 18),
 (8, 8),
 (0, 20),
 (13, 21),
 (1, 14),
 (10, 11),
 (9, 13),
 (31, 29),
 (16, 11),
 (25, 24),
 (4, 4),
 (3, 20),
 (5, 20),
 (4, 17),
 (11, 11),
 (2, 10),
 (4, 16),
 (3, 7),
 (15, 6),
 (11, 11),
 (8, 7),
 (10, 9),
 (2, 2),
 (0, 12),
 

In [106]:
import pandas as pd
pd.DataFrame(resi_search).describe()

,0,1
count,1000.000000,1000.000000
mean,8.878000,9.300000
std,6.124592,6.173177
min,0.000000,0.000000
25%,4.000000,4.000000
50%,8.000000,9.000000
75%,13.000000,14.000000
max,31.000000,29.000000


In [57]:
each = json_c[1000]
each

{'best_match': 'let k := (has_reflect l).arrow',
 'score': 0.3333333333333333,
 'real': 'haveI := has_reflect',
 'all': ['exact (has_lift_',
  'rw [mirror_eq_rel.2 _]',
  "rcases exists_mul_eq_mul_aux' (reflect tactic.rcases_patt).exists with ⟨b⟩|rfl",
  'exact is_colimit.reflect_rel (TM1.reflect l)',
  "exact pnat.reflect.reflect' (λ (e : l = 0), by rw [this, (mt (reflect e) l, eq_comm)] )",
  'induction n with n IH',
  'exact (has',
  'exact rfl',
  'rw ← reflection_patt',
  'apply_instance',
  'induction l with n l ih',
  'exact reflect_reflect tactic.rcases_patt',
  'induction n with n l IH',
  'exact refl_reflect_aux l l',
  'exact (reverse_rec _ _',
  'exact [rcases_patt, diff_empty.elim]',
  'exact (reflect_eq_self',
  'conv { to_lhs | (_root_.has_reflect tactic.rcases_patt) <immigrant} { rw ← l, exact (reflect (mt_trans (eq_iff_iff.mp aux.trans_le this))) }',
  'let k := (has_reflect l).arrow',
  'exact reflection_patt_reflect tactic.rcases_patt',
  'induction ih',
  'exact (l.

In [58]:
test = each["prompt"].replace("GOAL","<GOAL>").replace("PROOFSTEP","<PROOFSTEP>")

In [59]:
results = [i for i in each["all"]]

In [84]:
l = []
for i in results+[each["real"]]:
    inputs = rk.tokenize(test, i, return_tensors='pt')
    inputs.to(torch.device("cuda:0"))
    score = rk(inputs).logits
    l.append((score.cpu().detach().numpy()[0][0],i))
    l.sort(key=lambda x:x[0])
    l.reverse()

In [89]:
l

[(1.0750328, 'exact (has_lift_t'),
 (0.51294786, 'exact (has_lift_'),
 (-0.49501634, 'rw ← reflection_patt'),
 (-0.5962713, 'induction l with n l IH'),
 (-0.88799334, 'let k := (has_reflect l).arrow'),
 (-0.8978929, 'exact reflect_reflect tactic.rcases_patt'),
 (-0.98143125, 'induction n with n IH'),
 (-1.0099363, 'apply_instance'),
 (-1.0381949, 'rcases l with (_ | ⟨x, hx⟩)'),
 (-1.0494938, 'induction n with n l IH'),
 (-1.0732956, 'rw ←_root_.reflect_comp'),
 (-1.0913246, 'substs l l'),
 (-1.0934861, 'rw [←reflect_out, ←reflect]'),
 (-1.1548848, 'exact (reflect_eq_self'),
 (-1.1796035, 'induction l with n l ih'),
 (-1.1827998, 'haveI := has_reflect'),
 (-1.2198576, 'exact (has'),
 (-1.2420163, 'induction ih'),
 (-1.2687602, 'exact reflection_patt_reflect tactic.rcases_patt'),
 (-1.3217957, 'rw [mirror_eq_rel.2 _]'),
 (-1.4752369, 'exact refl_reflect_aux l l'),
 (-1.5429989, 'exact (l.'),
 (-1.6189396,
  "rcases exists_mul_eq_mul_aux' (reflect tactic.rcases_patt).exists with ⟨b⟩|rfl")

In [86]:
l_e = [i[1] for i in l]

In [87]:
l_e

['exact (has_lift_t',
 'exact (has_lift_',
 'rw ← reflection_patt',
 'induction l with n l IH',
 'let k := (has_reflect l).arrow',
 'exact reflect_reflect tactic.rcases_patt',
 'induction n with n IH',
 'apply_instance',
 'rcases l with (_ | ⟨x, hx⟩)',
 'induction n with n l IH',
 'rw ←_root_.reflect_comp',
 'substs l l',
 'rw [←reflect_out, ←reflect]',
 'exact (reflect_eq_self',
 'induction l with n l ih',
 'haveI := has_reflect',
 'exact (has',
 'induction ih',
 'exact reflection_patt_reflect tactic.rcases_patt',
 'rw [mirror_eq_rel.2 _]',
 'exact refl_reflect_aux l l',
 'exact (l.',
 "rcases exists_mul_eq_mul_aux' (reflect tactic.rcases_patt).exists with ⟨b⟩|rfl",
 'exact [rcases_patt, diff_empty.elim]',
 'exact is_colimit.reflect_rel (TM1.reflect l)',
 "exact pnat.reflect.reflect' (λ (e : l = 0), by rw [this, (mt (reflect e) l, eq_comm)] )",
 'induction n with n IH generalizing l',
 'coe : listΣ (append_nil : ℕ →',
 'exact (reverse_rec _ _',
 'induction n with n l IH generalizing l

In [76]:
each["real"]

'haveI := has_reflect'

In [79]:
l_e.index(each["real"])

15

In [80]:
l_e.index(each["best_match"])

4

In [88]:
each["best_match"]

'let k := (has_reflect l).arrow'

In [ ]:
"GOAL α : Type u,\t_inst_1 : inhabited α,\tb : buffer α,\ti : ℕ,\th : i < b.size\t⊢ b.read ⟨i, h⟩ = b.read i\n <PROOFSTEP> cases "

In [36]:
ls files_upload

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
data_test.jsonl           data_train_prepared.jsonl  wandb/
data_test_prepared.jsonl  data_valid.jsonl
data_train.jsonl          data_valid_prepared.jsonl


In [1]:
import json

with open('files_upload/data_train_prepared.jsonl', 'r') as json_file:
    json_c_all = [json.loads(i) for i in list(json_file)]

In [65]:
len(json_c_all)

168590

In [5]:
from transformers import AutoTokenizer

In [6]:

hf_tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M",    
                            bos_token="<|startoftext|>",
                            eos_token="<|endoftext|>",
                            pad_token="<|pad|>")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
from transformers import GPTNeoModel

In [8]:
data_args = {
    "train_dir":"data/train",
    "train_path":"data/train/train.json",
    "pred_dir":"data/dev",
    "pred_path":"data/train/dev.json",
    "train_group_size":8
}

In [9]:
from reranker.data import GroupedTrainDataset
train_dataset = GroupedTrainDataset(
    data_args, data_args["train_path"], 
    tokenizer=hf_tokenizer)

Using custom data configuration default-21c912046bcb6d4c
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-21c912046bcb6d4c/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
train_dataset[0]

[{'pid': '6575927600422562800', 'passage': [31653, 685, 29705, 238, 23611, 13, 23661, 357, 421, 313, 13, 28015, 62, 27363, 62, 28015, 62, 1659, 62, 27363, 4808, 828, 17804, 238, 23611, 13, 12543, 62, 15699, 62, 16]}, {'pid': '928431572601992639', 'passage': [31653, 685, 29705, 238, 23611, 13, 23661, 357, 421, 313, 13, 28015, 62, 27363, 62, 28015, 62, 1659, 62, 27363, 4808, 828, 17804, 238, 23611, 13, 26282, 62, 261, 6, 62]}, {'pid': '7552767570238416096', 'passage': [31653, 685, 29705, 238, 23611, 13, 23661, 357, 421, 313, 13, 28015, 62, 27363, 62, 28015, 4808, 4808, 828, 17804, 238, 23611, 13, 28015, 62, 27363, 62, 28015, 11, 17804, 238]}, {'pid': '-2234788984240462035', 'passage': [1069, 529]}, {'pid': '-4411941448410955259', 'passage': [31653, 685, 29705, 238, 23611, 13, 23661, 357, 421, 313, 13, 28015, 62, 27363, 62, 28015, 62, 1659, 62, 27363, 4808, 828, 17804, 238, 23611, 13, 23661, 357, 421, 313, 13]}, {'pid': '3122899436888549272', 'passage': [742]}, {'pid': '-88399759144270808

TypeError: TextInputSequence must be str

In [21]:
from reranker import RerankerTrainer

In [22]:
from reranker.data import GroupedTrainDataset, PredictionDataset, GroupCollator

In [23]:
trainer = RerankerTrainer(
        model=model,
        train_dataset=train_dataset,
        data_collator=GroupCollator(hf_tokenizer)
    )

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [24]:
# training_args = TrainingArguments("test-trainer",fp16=True)

In [25]:
# training_args

In [26]:
# from transformers import TrainingArguments

In [27]:
trainer.train()

***** Running training *****
  Num examples = 6058
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2271


AttributeError: 'dict' object has no attribute 'max_len'

In [51]:
[i for i in hf_model(**inputs).last_hidden_state]

['last_hidden_state', 'past_key_values']

In [79]:
hf_model.cuda()

GPTNeoModel(
  (wte): Embedding(50259, 768)
  (wpe): Embedding(2048, 768)
  (drop): Dropout(p=0, inplace=False)
  (h): ModuleList(
    (0): GPTNeoBlock(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPTNeoAttention(
        (attention): GPTNeoSelfAttention(
          (attn_dropout): Dropout(p=0, inplace=False)
          (resid_dropout): Dropout(p=0, inplace=False)
          (k_proj): Linear(in_features=768, out_features=768, bias=False)
          (v_proj): Linear(in_features=768, out_features=768, bias=False)
          (q_proj): Linear(in_features=768, out_features=768, bias=False)
          (out_proj): Linear(in_features=768, out_features=768, bias=True)
        )
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPTNeoMLP(
        (c_fc): Linear(in_features=768, out_features=3072, bias=True)
        (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        (act): NewGELUActivation()
        (drop

In [30]:
json_c_all[0]

NameError: name 'json_c_all' is not defined

In [70]:
inputs.to(torch.device("cuda:0"))

{'input_ids': tensor([[11230,  1847,   371,  1058,  5994,   334,    11,   197,    43,  1058,
          5994,   410,    11,   197,    44,  1058,  5994,   266,    11,   197,
            62,  8625,    62,    16,  1058,   725,    62,  1806,   371,    11,
           197,    62,  8625,    62,    17,  1058,  6486,    62,  1806,   406,
            11,   197,    62,  8625,    62,    18,  1058,  6486,    62,   282,
         29230,   371,   406,    11,   197,    62,  8625,    62,    19,  1058,
           751,    62,  9503,    62,  8094,   337,    11,   197,    62,  8625,
            62,    20,  1058,  8265,   371,   337,    11,   197,    62,  8625,
            62,    21,  1058,  6486,    62,  1806,    62, 21412,   406,   337,
            11,   197,    62,  8625,    62,    22,  1058,  6486,    62, 21412,
           371,   406,   337,    11,   197,    45,   399,     6,  1058,  6486,
            62,  7266, 21412,   371,   406,   337,    11,   197,    71,  1058,
         24935,    45,   796, 24935,  

In [56]:
import numpy as np

In [80]:
%%time
inputs = rk.tokenize("GOAL R : Type u,\tL : Type v,\tM : Type w,\t_inst_1 : comm_ring R,\t_inst_2 : lie_ring L,\t_inst_3 : lie_algebra R L,\t_inst_4 : add_comm_group M,\t_inst_5 : module R M,\t_inst_6 : lie_ring_module L M,\t_inst_7 : lie_module R L M,\tN N' : lie_submodule R L M,\th : ↑N = ↑N',\tm : M\t⊢ m ∈ N ↔ m ∈ N'\n <PROOFSTEP> ", "rw [\u2190 mem_coe_submodule, h]\n", return_tensors='pt')
inputs.to(torch.device("cuda:0"))
torch.mean(hf_model(**inputs).last_hidden_state,axis=1)

CPU times: user 34.9 ms, sys: 20.5 ms, total: 55.3 ms
Wall time: 51.9 ms


tensor([[ 2.5791e-01,  2.2195e-01,  6.1754e-01, -4.1989e-01, -1.1693e+00,
         -4.2416e-01, -1.2921e-02, -6.3473e-02,  1.8195e-01, -2.2530e-02,
          6.7583e-01, -5.1123e-01,  8.9316e-02,  7.8981e-01, -2.5015e-01,
          1.9408e-01,  6.3822e-01,  9.7251e-01, -7.8701e-02, -6.0445e-01,
         -9.8325e-01,  2.0833e-01, -1.9024e+00,  9.8087e-01,  2.8801e-02,
         -5.0874e-01,  1.5393e+00, -3.0677e-01,  1.4771e-01, -1.6751e+00,
         -1.6980e+00,  8.8647e-01,  8.6847e-01,  8.0590e-01, -1.2548e+00,
         -1.8401e-02, -5.4263e-01, -6.5954e-01, -3.4339e-01, -4.9939e-01,
          5.0794e-01,  4.8216e-01,  6.5291e-02,  6.9481e-02,  9.3943e-01,
          1.1056e-01, -9.0834e-01,  5.9255e-01,  6.8692e-01,  2.9180e-01,
          3.6947e-01,  7.4422e-01,  1.4877e-01,  4.4374e-01,  1.0055e+00,
         -7.2418e-01,  1.5478e-01,  2.2642e-01,  2.0349e-01,  6.3065e-01,
         -4.8900e-01, -1.1986e+00, -2.5607e-01,  9.0008e-01,  1.2902e-01,
          5.3663e+00, -1.3429e-01,  7.

In [75]:
torch.cat((inputs,inputs))

TypeError: expected Tensor as element 0 in argument 0, but got BatchEncoding

In [76]:
hf_model

GPTNeoModel(
  (wte): Embedding(50259, 768)
  (wpe): Embedding(2048, 768)
  (drop): Dropout(p=0, inplace=False)
  (h): ModuleList(
    (0): GPTNeoBlock(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPTNeoAttention(
        (attention): GPTNeoSelfAttention(
          (attn_dropout): Dropout(p=0, inplace=False)
          (resid_dropout): Dropout(p=0, inplace=False)
          (k_proj): Linear(in_features=768, out_features=768, bias=False)
          (v_proj): Linear(in_features=768, out_features=768, bias=False)
          (q_proj): Linear(in_features=768, out_features=768, bias=False)
          (out_proj): Linear(in_features=768, out_features=768, bias=True)
        )
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPTNeoMLP(
        (c_fc): Linear(in_features=768, out_features=3072, bias=True)
        (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        (act): NewGELUActivation()
        (drop